The purpose of this notebook is mainly to check the correctness of new real values spherical harmonics


In [1]:
import numpy as np
import ase.io as ase_io
from ase import Atoms

import pyximport; pyximport.install()
import nice_utilities
import naive, radial_basis, spherical_coefficients, spherical_harmonics
import ClebschGordan, test_utilities

/home/pozdn/.local/lib/python3.6/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/pozdn/methane_nice/collecting_standard_features/cleaned/naive.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [2]:
structure = ase_io.read('structures.xyz', index = '0:1')[0]

In [3]:
env = structure.positions[1:]
print(env)

[[ 1.1957718   0.69582632 -1.5377768 ]
 [-0.07590599  1.04620262  1.95058034]
 [-1.10813614  1.91728119  1.81155538]
 [ 1.39764218 -0.98903322 -0.05350022]]


In [4]:
clebsch = ClebschGordan.ClebschGordan(5)

In [5]:
coef_calc = spherical_coefficients.SphericalCoefficientsCalculator(
            5, 5, 5.0, radial_basis.simple_sin)

In [6]:
coefs = coef_calc.transform_single([np.array([0, 0, 0, 0]), np.array([0, 0, 0, 0]), env], 4, 1)[0]
print(coefs.shape)

(5, 6, 11)


In [7]:
ps = naive.compute_powerspectrum(coefs, 5)
print(ps.shape)
print(ps[0:10])

(90,)
[0.57517046 0.25849786 0.73582562 1.77225341 0.83432825 1.61267425
 0.19121661 0.03525373 0.05378041 0.4312143 ]


In [8]:
for _ in range(10):
    rotated = test_utilities.rotate_env(env)
    coefs_rotated = coef_calc.transform_single([np.array([0, 0, 0, 0]), np.array([0, 0, 0, 0]), rotated], 4, 1)[0]
    ps_new = naive.compute_powerspectrum(coefs_rotated, 5)
    print(np.sum(np.abs(ps - ps_new)))

5.362377208939506e-14
6.670705654521214e-14
4.0316708305176974e-14
2.6700863742235015e-14
1.7501625149130007e-14
6.934557095217286e-14
6.108308303609533e-14
5.3463310167867206e-14
6.110389971780705e-14
2.447521352255677e-14


In [10]:
bs = naive.compute_bispectrum(clebsch.precomputed, coefs, 5)

print(bs.shape)
print(bs[0:20])

(3885,)
[ 6.16893026e-01  2.77249159e-01  7.89202027e-01  1.90081175e+00
  8.94849983e-01  1.72965680e+00  2.77249159e-01 -1.60069877e-01
  0.00000000e+00 -9.09055389e-02  7.04151277e-02  2.03287907e-18
  4.95600036e-01 -4.18858479e-01  8.15320034e-17  3.70889818e-01
 -3.27094074e-01 -1.56125113e-17  1.35626988e-02 -1.22679227e-02]


In [12]:
for _ in range(10):
    bs = naive.compute_bispectrum(clebsch.precomputed, coefs, 5)
    
    rotated = test_utilities.rotate_env(env)
    coefs_rotated = coef_calc.transform_single([np.array([0, 0, 0, 0]),np.array([0, 0, 0, 0]), rotated], 4, 1)[0]
   
    bs_new = naive.compute_bispectrum(clebsch.precomputed, coefs_rotated, 5)
   
    print(np.sum(np.abs(bs - bs_new)))
    

1.023439408934905e-12
9.29470101443086e-13
1.0196575708703662e-12
6.922870614825825e-13
1.067886795578501e-12
4.3503894047801785e-13
8.098594270985655e-13
5.681201250316441e-13
8.967266233073137e-13
2.797845978747842e-13


In [15]:
for _ in range(1):
    bs = naive.compute_bispectrum(clebsch.precomputed, coefs, 5)
    
    inverted = -env
    coefs_inverted = coef_calc.transform_single([np.array([0, 0, 0, 0]),np.array([0, 0, 0, 0]), inverted], 4, 1)[0]
   
    bs_new = naive.compute_bispectrum(clebsch.precomputed, coefs_inverted, 5)
   
    print(np.sum(np.abs(bs - bs_new)))
    

97.4751609942672


In [16]:
for _ in range(1):
    bs = naive.compute_bispectrum(clebsch.precomputed, coefs, 5, only_even = True)
    
    inverted = -env
    coefs_inverted = coef_calc.transform_single([np.array([0, 0, 0, 0]),np.array([0, 0, 0, 0]), inverted], 4, 1)[0]
   
    bs_new = naive.compute_bispectrum(clebsch.precomputed, coefs_inverted, 5, only_even = True)
   
    print(np.sum(np.abs(bs - bs_new)))
    

2.2128873982795588e-13
